In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load the CSV file
file_path = '/Users/shashinimashi/Desktop/Semester 3/Thesis/Analysis/Repo/Research_OAI/Research_OAI/transcripts.csv'  
df = pd.read_csv(file_path)

# Check if the 'Transcription' column exists
if 'Transcription' not in df.columns:
    raise ValueError("The 'Transcription' column is missing in the dataset.")

# Load the pretrained embedding model
embedding_model = SentenceTransformer('all-mpnet-base-v2')

# Generate embeddings for the Transcription column
df['Transcription_Embeddings'] = df['Transcription'].apply(
    lambda text: embedding_model.encode(text, convert_to_tensor=True)
)

# Save the updated DataFrame to a CSV file
output_path = 'transcription_embeddings.csv'
df.to_csv(output_path, index=False)

print(f"Embeddings file saved to {output_path}")


#open with pickel
# Load train response embeddings
#with open('train_response_embeddings.pkl', 'rb') as f:
    loaded_train_response_embeddings = pickle.load(f)

# Load validation response embeddings
#with open('val_response_embeddings.pkl', 'rb') as f:
    loaded_val_response_embeddings = pickle.load(f)

# Load test response embeddings
#with open('test_response_embeddings.pkl', 'rb') as f:
#   loaded_test_response_embeddings = pickle.load(f)


In [ ]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df_cleaned, test_size=0.2, random_state=42)  # 80% training data
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)  # 10% validation, 10% test

print(f"Training Set: {len(train_df)}")
print(f"Validation Set: {len(val_df)}")
print(f"Test Set: {len(test_df)}")


In [ ]:
# Print the column names of each dataframe
print(f"train_df columns: {train_df.columns}")
print(f"val_df columns: {val_df.columns}")
print(f"test_df columns: {test_df.columns}")

In [ ]:
# Check the first few rows of 'Responses' column in each dataset
print(train_df['Responses'].head())
print(val_df['Responses'].head())
print(test_df['Responses'].head())

In [ ]:
# Check how many valid responses are left
print(f"Valid Train Responses: {len([r for r in train_responses if r != ''])}")
print(f"Valid Validation Responses: {len([r for r in val_responses if r != ''])}")
print(f"Valid Test Responses: {len([r for r in test_responses if r != ''])}")

In [ ]:
# Check the first few responses after cleaning
print(train_df['Responses'].head())
print(val_df['Responses'].head())
print(test_df['Responses'].head())

# Check if there are any empty or None values
empty_train_responses = train_df['Responses'].apply(lambda x: len(x) == 0 if isinstance(x, list) else True)
empty_val_responses = val_df['Responses'].apply(lambda x: len(x) == 0 if isinstance(x, list) else True)
empty_test_responses = test_df['Responses'].apply(lambda x: len(x) == 0 if isinstance(x, list) else True)

print(f"Empty Train Responses: {empty_train_responses.sum()}")
print(f"Empty Validation Responses: {empty_val_responses.sum()}")
print(f"Empty Test Responses: {empty_test_responses.sum()}")

In [ ]:
from sentence_transformers import SentenceTransformer
import torch
import numpy as np

# Initialize the SentenceTransformer model (all-mpnet-base-v2)
model = SentenceTransformer('all-mpnet-base-v2')

# Function to generate sentence embeddings
def get_sentence_embeddings(text_list):
    # Generate sentence embeddings for the text list
    return model.encode(text_list, convert_to_tensor=True)

# Flatten the list of sentences (Responses are lists of sentences, so flatten them into one list)
def flatten_responses(responses):
    return [sentence for response in responses for sentence in response]

# For the training, validation, and test sets, flatten the responses before getting embeddings
train_flat_responses = flatten_responses(train_df['Responses'].tolist())
val_flat_responses = flatten_responses(val_df['Responses'].tolist())
test_flat_responses = flatten_responses(test_df['Responses'].tolist())

# Get embeddings for the flattened responses
train_embeddings = get_sentence_embeddings(train_flat_responses)
val_embeddings = get_sentence_embeddings(val_flat_responses)
test_embeddings = get_sentence_embeddings(test_flat_responses)

# Now, group by ResponseID (Mean Pooling) - assuming 'ResponseID' is already available
# Grouping the responses by ResponseID and applying mean pooling to get one embedding per response
def group_by_response_id(df, embeddings):
    response_embeddings = {}
    
    # Iterate through unique ResponseIDs
    for response_id in df['ResponseID'].unique():
        # Get sentences for the given ResponseID
        sentences = df[df['ResponseID'] == response_id]['Responses'].tolist()
        
        # Flatten the sentences for the ResponseID
        sentences_flat = flatten_responses(sentences)
        
        # Generate sentence embeddings for the flattened sentences
        sentences_embeddings = model.encode(sentences_flat, convert_to_tensor=True)
        
        # Apply mean pooling using PyTorch's .mean() method along the right axis
        response_embeddings[response_id] = sentences_embeddings.mean(dim=0)  # Mean Pooling
    
    return response_embeddings

# Example for training, validation, and test sets
train_response_embeddings = group_by_response_id(train_df, train_embeddings)
val_response_embeddings = group_by_response_id(val_df, val_embeddings)
test_response_embeddings = group_by_response_id(test_df, test_embeddings)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score
import numpy as np
from sentence_transformers import SentenceTransformer

# Initialize the SentenceTransformer model (all-mpnet-base-v2)
model = SentenceTransformer('all-mpnet-base-v2')

# Function to calculate cosine similarity
def calculate_cosine_similarity(query_embedding, response_embeddings):
    similarities = {}
    for response_id, embedding in response_embeddings.items():
        # Ensure both embeddings are moved to CPU and converted to NumPy
        similarity = cosine_similarity(
            query_embedding.cpu().numpy(),
            embedding.cpu().numpy().reshape(1, -1)
        )[0][0]
        similarities[response_id] = similarity
    return similarities

# Example threshold and fallback count
THRESHOLD = 0.2
TOP_N = 5  # Fallback: Select top N responses if no responses pass the threshold

# Normalize keyword embeddings
keyword_embeddings_cpu = normalize(keyword_embeddings.cpu().numpy(), axis=1)

# Calculate cosine similarity between keyword embeddings and response embeddings
response_similarities = {}  # Store similarities for all responses
for response_id, embedding in train_response_embeddings.items():
    # Normalize response embeddings
    response_embedding = normalize(embedding.cpu().numpy().reshape(1, -1))
    
    # Calculate cosine similarity for all keywords
    max_similarity = max(
        cosine_similarity(keyword_embeddings_cpu, response_embedding).flatten()
    )
    response_similarities[response_id] = max_similarity

# Filter responses based on threshold
filtered_responses = {
    response_id: sim for response_id, sim in response_similarities.items() if sim >= THRESHOLD
}

# Fallback: If no responses pass the threshold, select the top N most similar responses
if not filtered_responses:
    print("\nNo responses passed the threshold. Using fallback mechanism.")
    filtered_responses = dict(sorted(response_similarities.items(), key=lambda x: x[1], reverse=True)[:TOP_N])

# Display filtered responses
print("\nFiltered Responses:")
print(filtered_responses)



In [ ]:
from sentence_transformers import SentenceTransformer
import torch
import numpy as np

# Initialize the SentenceTransformer model (all-mpnet-base-v2)
model = SentenceTransformer('all-mpnet-base-v2')

# Function to generate sentence embeddings
def get_sentence_embeddings(text_list):
    # Generate sentence embeddings for the text list
    return model.encode(text_list, convert_to_tensor=True)

# Flatten the list of sentences (Responses are lists of sentences, so flatten them into one list)
def flatten_responses(responses):
    return [sentence for response in responses for sentence in response]

# For the training, validation, and test sets, flatten the responses before getting embeddings
train_flat_responses = flatten_responses(train_df['Responses'].tolist())
val_flat_responses = flatten_responses(val_df['Responses'].tolist())
test_flat_responses = flatten_responses(test_df['Responses'].tolist())

# Get embeddings for the flattened responses
train_embeddings = get_sentence_embeddings(train_flat_responses)
val_embeddings = get_sentence_embeddings(val_flat_responses)
test_embeddings = get_sentence_embeddings(test_flat_responses)

# Now, group by ResponseID (Mean Pooling) - assuming 'ResponseID' is already available
# Grouping the responses by ResponseID and applying mean pooling to get one embedding per response
def group_by_response_id(df, embeddings):
    response_embeddings = {}
    
    # Iterate through unique ResponseIDs
    for response_id in df['ResponseID'].unique():
        # Get sentences for the given ResponseID
        sentences = df[df['ResponseID'] == response_id]['Responses'].tolist()
        
        # Flatten the sentences for the ResponseID
        sentences_flat = flatten_responses(sentences)
        
        # Generate sentence embeddings for the flattened sentences
        sentences_embeddings = model.encode(sentences_flat, convert_to_tensor=True)
        
        # Apply mean pooling using PyTorch's .mean() method along the right axis
        response_embeddings[response_id] = sentences_embeddings.mean(dim=0)  # Mean Pooling
    
    return response_embeddings

# Example for training, validation, and test sets
train_response_embeddings = group_by_response_id(train_df, train_embeddings)
val_response_embeddings = group_by_response_id(val_df, val_embeddings)
test_response_embeddings = group_by_response_id(test_df, test_embeddings)
